In [ ]:
from google.colab import drive
drive.mount('/content/drive')
datadir = "/content/drive/MyDrive/CS441/FP2/data/snippets"
save_dir = "/content/drive/My Drive/CS441/FP2"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchcodec

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchaudio
from transformers import (
    DistilBertTokenizerFast,
    DistilBertModel,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Model
)
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

In [ ]:
class PodcastDataset(Dataset):
    def __init__(self, csv_path, audio_root, tokenizer, feature_extractor, max_audio_len=5.0):
        self.data = pd.read_csv(csv_path)
        self.audio_root = audio_root
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.max_audio_len = max_audio_len

        genres = sorted(self.data["genre"].unique())
        self.label_map = {g: i for i, g in enumerate(genres)}
        print(f"📊 Loaded {len(self.data)} samples, {len(genres)} classes: {genres}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # ----- Audio -----
        audio_path = os.path.join(self.audio_root, "snippets", row["genre"], row["path"])
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio not found: {audio_path}")

        wav, sr = torchaudio.load(audio_path)

        # Switched to mono
        if wav.size(0) > 1:
            wav = wav.mean(dim=0, keepdim=True)

        # resample to 16kHz
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            wav = resampler(wav)
            sr = 16000

        # Crop or padding to a fixed length
        target_len = int(self.max_audio_len * sr)
        if wav.size(1) > target_len:
            wav = wav[:, :target_len]
        elif wav.size(1) < target_len:
            pad_len = target_len - wav.size(1)
            wav = torch.nn.functional.pad(wav, (0, pad_len))

        # Normalized audio
        wav = wav / (wav.abs().max() + 1e-8)

        audio_features = self.feature_extractor(
            wav.squeeze().numpy(), sampling_rate=sr, return_tensors="pt"
        )

        # ----- Text -----
        text = row["transcript"]
        max_len = 256

        if pd.isna(text) or text.strip() == "":
            use_text = False
            input_ids = torch.zeros(max_len, dtype=torch.long)
            attention_mask = torch.zeros(max_len, dtype=torch.long)
        else:
            use_text = True
            encoded_text = self.tokenizer(
                text,
                truncation=True,
                padding="max_length",
                max_length=max_len,
                return_tensors="pt"
            )
            input_ids = encoded_text["input_ids"].squeeze(0)
            attention_mask = encoded_text["attention_mask"].squeeze(0)

        label = self.label_map[row["genre"]]

        return {
            "audio_values": audio_features["input_values"].squeeze(0),
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(label),
            "use_text": torch.tensor(use_text, dtype=torch.bool)
        }

In [ ]:
class ImprovedAudioTextClassifier(nn.Module):
    def __init__(self, num_classes, freeze_pretrained=False):  # unfreeze
        super().__init__()

        # Audio encoder
        self.audio_encoder = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        # Text encoder
        self.text_encoder = DistilBertModel.from_pretrained("distilbert-base-uncased")

        if freeze_pretrained:
            for param in self.audio_encoder.parameters():
                param.requires_grad = False
            for param in self.text_encoder.parameters():
                param.requires_grad = False
        else:
            # Only fine-tune the last few layers
            # Freeze the first 4 layers of Wav2Vec2
            for i, layer in enumerate(self.audio_encoder.encoder.layers):
                if i < 4:
                    for param in layer.parameters():
                        param.requires_grad = False

            # Freeze the first 2 layers of DistilBERT
            for i, layer in enumerate(self.text_encoder.transformer.layer):
                if i < 2:
                    for param in layer.parameters():
                        param.requires_grad = False

        # Improved Projection Layer (with BatchNorm and Dropout)
        self.audio_proj = nn.Sequential(
            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.text_proj = nn.Sequential(
            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        # Attention Fusion Layer
        self.fusion_attention = nn.Sequential(
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.Softmax(dim=1)
        )

        # Improved classifier
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_audio, input_ids=None, attention_mask=None, use_text=None):
        # ===== audio features =====
        audio_out = self.audio_encoder(input_audio).last_hidden_state
        audio_feat = audio_out.mean(dim=1)  # [batch_size, 768]
        audio_feat = self.audio_proj(audio_feat)  # [batch_size, 512]

        # ===== text features =====
        batch_size = audio_feat.size(0)
        text_feat = torch.zeros(batch_size, 512, device=audio_feat.device, dtype=audio_feat.dtype)

        if use_text is not None:
            text_mask = use_text.bool()

            if text_mask.any():
                valid_input_ids = input_ids[text_mask]
                valid_attention_mask = attention_mask[text_mask]

                text_out = self.text_encoder(
                    input_ids=valid_input_ids,
                    attention_mask=valid_attention_mask
                ).last_hidden_state

                text_feat_valid = text_out.mean(dim=1)
                text_feat_valid = self.text_proj(text_feat_valid)
                text_feat[text_mask] = text_feat_valid

        # ===== Attention Fusion =====
        fused = torch.cat([audio_feat, text_feat], dim=1)  # [batch_size, 1024]

        # Calculate attention weights
        attn_weights = self.fusion_attention(fused)  # [batch_size, 2]

        # Weighted fusion
        weighted_audio = audio_feat * attn_weights[:, 0:1]
        weighted_text = text_feat * attn_weights[:, 1:2]
        final_feat = weighted_audio + weighted_text  # [batch_size, 512]

        logits = self.classifier(final_feat)
        return logits

In [ ]:
# Train / Eval
# =====================================
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in loader:
            logits = model(
                input_audio=batch["audio_values"].to(device),
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                use_text=batch["use_text"].to(device)
            )
            preds = logits.argmax(dim=1)
            labels = batch["labels"].to(device)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    return accuracy, all_preds, all_labels

In [ ]:
def train_model(train_csv, val_csv, audio_root, batch_size=8, lr=2e-5, epochs=10, warmup_epochs=2):
    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

    train_ds = PodcastDataset(train_csv, audio_root, tokenizer, feature_extractor)
    val_ds = PodcastDataset(val_csv, audio_root, tokenizer, feature_extractor)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)

    num_classes = len(train_ds.label_map)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ImprovedAudioTextClassifier
    model = ImprovedAudioTextClassifier(num_classes=num_classes, freeze_pretrained=False).to(device)

    # different learning rate
    pretrained_params = []
    new_params = []
    for name, param in model.named_parameters():
        if 'audio_encoder' in name or 'text_encoder' in name:
            pretrained_params.append(param)
        else:
            new_params.append(param)

    optimizer = AdamW([
        {'params': pretrained_params, 'lr': lr * 0.1},  # Pre-training layer with smaller learning rate
        {'params': new_params, 'lr': lr}
    ])

    # Add a learning rate scheduler
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)

    criterion = nn.CrossEntropyLoss()
    best_acc = 0
    patience = 5
    no_improve = 0

    print(f"\n🚀 Starting training...")
    print(f"Device: {device}")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}\n")

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in pbar:
            optimizer.zero_grad()

            logits = model(
                input_audio=batch["audio_values"].to(device),
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                use_text=batch["use_text"].to(device)
            )
            loss = criterion(logits, batch["labels"].to(device))

            if torch.isnan(loss) or torch.isinf(loss):
                print(f"\n⚠️ Warning: Invalid loss detected, skipping batch")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / len(train_loader)
        val_acc, _, _ = evaluate(model, val_loader, device)

        print(f">>> Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({
                'model_state_dict': model.state_dict(),
                'epoch': epoch,
                'val_acc': val_acc,
                'label_map': train_ds.label_map
            }, "best_multimodal_model.pt")
            print(f"✔ Saved best model (Acc: {val_acc:.4f})")
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"\n⚠️ Early stopping after {epoch+1} epochs (no improvement for {patience} epochs)")
                break

        scheduler.step()

    print(f"\n🎉 Training complete. Best Val Acc = {best_acc:.4f}")
    return model, train_ds.label_map

In [ ]:
# Test
# =====================================
def test_model(model_path, test_csv, audio_root, label_map, batch_size=8):
    """Evaluate model performance in test dataset"""
    from sklearn.metrics import classification_report, confusion_matrix
    import numpy as np

    print("\n" + "="*50)
    print("📊 Testing on Test Set")
    print("="*50 + "\n")

    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

    test_ds = PodcastDataset(test_csv, audio_root, tokenizer, feature_extractor)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)

    num_classes = len(label_map)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ImprovedAudioTextClassifier(num_classes=num_classes, freeze_pretrained=False).to(device)

    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✔ Loaded best model from epoch {checkpoint['epoch']+1} (Val Acc: {checkpoint['val_acc']:.4f})\n")

    test_acc, all_preds, all_labels = evaluate(model, test_loader, device)

    idx_to_label = {v: k for k, v in label_map.items()}
    class_names = [idx_to_label[i] for i in range(num_classes)]

    print(f"{'='*50}")
    print(f"🎯 Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"{'='*50}\n")

    print("📋 Classification Report:")
    print("-" * 50)
    print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))

    print("\n📊 Confusion Matrix:")
    print("-" * 50)
    cm = confusion_matrix(all_labels, all_preds)

    header = "True\\Pred  " + "  ".join([f"{name[:8]:>8}" for name in class_names])
    print(header)
    print("-" * len(header))
    for i, row in enumerate(cm):
        row_str = f"{class_names[i][:10]:<10} " + "  ".join([f"{val:>8}" for val in row])
        print(row_str)

    print("\n📈 Per-Class Accuracy:")
    print("-" * 50)
    class_correct = cm.diagonal()
    class_total = cm.sum(axis=1)
    for i, name in enumerate(class_names):
        acc = class_correct[i] / class_total[i] if class_total[i] > 0 else 0
        print(f"{name:20s}: {acc:.4f} ({class_correct[i]:3d}/{class_total[i]:3d})")

    print("\n🔀 Most Confused Class Pairs:")
    print("-" * 50)
    confusion_pairs = []
    for i in range(num_classes):
        for j in range(num_classes):
            if i != j and cm[i, j] > 0:
                confusion_pairs.append((cm[i, j], class_names[i], class_names[j]))

    confusion_pairs.sort(reverse=True)
    for count, true_class, pred_class in confusion_pairs[:10]:
        print(f"{true_class:15s} → {pred_class:15s}: {count:3d} times")

    print("\n" + "="*50)
    print("✅ Testing Complete!")
    print("="*50)

    return test_acc, all_preds, all_labels

In [ ]:
# Run training and testing
if __name__ == "__main__":
    datadir = "/content/drive/My Drive/CS441/FP2/data/"

    # train
    model, label_map = train_model(
        train_csv=os.path.join(datadir, "train_transcripts.csv"),
        val_csv=os.path.join(datadir, "val_transcripts.csv"),
        audio_root=datadir,
        batch_size=8,
        lr=2e-5,
        epochs=20,
        warmup_epochs=2
    )

    # evaluate
    test_acc, test_preds, test_labels = test_model(
        model_path="best_multimodal_model.pt",
        test_csv=os.path.join(datadir, "test_transcripts.csv"),
        audio_root=datadir,
        label_map=label_map,
        batch_size=8
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📊 Loaded 1080 samples, 6 classes: ['business', 'comedy', 'education', 'news', 'religion', 'sports']
📊 Loaded 240 samples, 6 classes: ['business', 'comedy', 'education', 'news', 'religion', 'sports']


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🚀 Starting training...
Device: cuda
Total parameters: 162,446,472
Trainable parameters: 119,919,240



Epoch 1/20: 100%|██████████| 135/135 [00:33<00:00,  4.00it/s, loss=1.6857]


...

>>> Epoch 19 | Train Loss: 0.8592 | Val Acc: 0.6083 | LR: 4.89e-08


Epoch 20/20: 100%|██████████| 135/135 [00:31<00:00,  4.29it/s, loss=1.0091]


>>> Epoch 20 | Train Loss: 0.8591 | Val Acc: 0.6042 | LR: 1.23e-08

🎉 Training complete. Best Val Acc = 0.6208

📊 Testing on Test Set

📊 Loaded 240 samples, 6 classes: ['business', 'comedy', 'education', 'news', 'religion', 'sports']


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✔ Loaded best model from epoch 16 (Val Acc: 0.6208)

🎯 Test Accuracy: 0.7750 (77.50%)

📋 Classification Report:
--------------------------------------------------
              precision    recall  f1-score   support

    business     0.6415    0.8500    0.7312        40
      comedy     0.7297    0.6750    0.7013        40
   education     0.6667    0.8500    0.7473        40
        news     0.8571    0.4500    0.5902        40
    religion     0.9756    1.0000    0.9877        40
      sports     0.8919    0.8250    0.8571        40

    accuracy                         0.7750       240
   macro avg     0.7938    0.7750    0.7691       240
weighted avg     0.7938    0.7750    0.7691       240


📊 Confusion Matrix:
--------------------------------------------------
True\Pred  business    comedy  educatio      news  religion    sports
---------------------------------------------------------------------
business         34         1         4         1         0         0
comedy      

Tested

In [ ]:
# Test Accuracy: 0.7875 (78.75%)
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchaudio
from transformers import (
    DistilBertTokenizerFast,
    DistilBertModel,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Model
)
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

# =====================================
# Dataset (改进版)
# =====================================
class PodcastDataset(Dataset):
    def __init__(self, csv_path, audio_root, tokenizer, feature_extractor, max_audio_len=5.0):
        self.data = pd.read_csv(csv_path)
        self.audio_root = audio_root
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.max_audio_len = max_audio_len  # 最大音频长度（秒）

        genres = sorted(self.data["genre"].unique())
        self.label_map = {g: i for i, g in enumerate(genres)}
        print(f"📊 Loaded {len(self.data)} samples, {len(genres)} classes: {genres}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # ----- Audio -----
        audio_path = os.path.join(self.audio_root, "snippets", row["genre"], row["path"])
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio not found: {audio_path}")

        wav, sr = torchaudio.load(audio_path)

        # 转为单声道
        if wav.size(0) > 1:
            wav = wav.mean(dim=0, keepdim=True)

        # 重采样到16kHz
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            wav = resampler(wav)
            sr = 16000

        # 裁剪或填充到固定长度
        target_len = int(self.max_audio_len * sr)
        if wav.size(1) > target_len:
            wav = wav[:, :target_len]
        elif wav.size(1) < target_len:
            pad_len = target_len - wav.size(1)
            wav = torch.nn.functional.pad(wav, (0, pad_len))

        # 归一化音频
        wav = wav / (wav.abs().max() + 1e-8)

        audio_features = self.feature_extractor(
            wav.squeeze().numpy(), sampling_rate=sr, return_tensors="pt"
        )

        # ----- Text -----
        text = row["transcript"]
        max_len = 256

        if pd.isna(text) or text.strip() == "":
            use_text = False
            input_ids = torch.zeros(max_len, dtype=torch.long)
            attention_mask = torch.zeros(max_len, dtype=torch.long)
        else:
            use_text = True
            encoded_text = self.tokenizer(
                text,
                truncation=True,
                padding="max_length",
                max_length=max_len,
                return_tensors="pt"
            )
            input_ids = encoded_text["input_ids"].squeeze(0)
            attention_mask = encoded_text["attention_mask"].squeeze(0)

        label = self.label_map[row["genre"]]

        return {
            "audio_values": audio_features["input_values"].squeeze(0),
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(label),
            "use_text": torch.tensor(use_text, dtype=torch.bool)
        }


# =====================================
# 改进的模型架构
# =====================================
class ImprovedAudioTextClassifier(nn.Module):
    def __init__(self, num_classes, freeze_pretrained=False):  # 改为不冻结
        super().__init__()

        # Audio encoder
        self.audio_encoder = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        # Text encoder
        self.text_encoder = DistilBertModel.from_pretrained("distilbert-base-uncased")

        if freeze_pretrained:
            for param in self.audio_encoder.parameters():
                param.requires_grad = False
            for param in self.text_encoder.parameters():
                param.requires_grad = False
        else:
            # 只微调最后几层
            # 冻结 Wav2Vec2 的前 8 层
            for i, layer in enumerate(self.audio_encoder.encoder.layers):
                if i < 4:
                    for param in layer.parameters():
                        param.requires_grad = False

            # 冻结 DistilBERT 的前 4 层
            for i, layer in enumerate(self.text_encoder.transformer.layer):
                if i < 2:
                    for param in layer.parameters():
                        param.requires_grad = False

        # 改进的投影层（带BatchNorm和Dropout）
        self.audio_proj = nn.Sequential(
            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.text_proj = nn.Sequential(
            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        # 注意力融合层（替代简单拼接）
        self.fusion_attention = nn.Sequential(
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.Softmax(dim=1)
        )

        # 改进的分类器
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_audio, input_ids=None, attention_mask=None, use_text=None):
        # ===== 音频特征 =====
        audio_out = self.audio_encoder(input_audio).last_hidden_state
        audio_feat = audio_out.mean(dim=1)  # [batch_size, 768]
        audio_feat = self.audio_proj(audio_feat)  # [batch_size, 512]

        # ===== 文本特征 =====
        batch_size = audio_feat.size(0)
        text_feat = torch.zeros(batch_size, 512, device=audio_feat.device, dtype=audio_feat.dtype)

        if use_text is not None:
            text_mask = use_text.bool()

            if text_mask.any():
                valid_input_ids = input_ids[text_mask]
                valid_attention_mask = attention_mask[text_mask]

                text_out = self.text_encoder(
                    input_ids=valid_input_ids,
                    attention_mask=valid_attention_mask
                ).last_hidden_state

                text_feat_valid = text_out.mean(dim=1)
                text_feat_valid = self.text_proj(text_feat_valid)
                text_feat[text_mask] = text_feat_valid

        # ===== 注意力融合（替代简单拼接）=====
        fused = torch.cat([audio_feat, text_feat], dim=1)  # [batch_size, 1024]

        # 计算注意力权重
        attn_weights = self.fusion_attention(fused)  # [batch_size, 2]

        # 加权融合
        weighted_audio = audio_feat * attn_weights[:, 0:1]
        weighted_text = text_feat * attn_weights[:, 1:2]
        final_feat = weighted_audio + weighted_text  # [batch_size, 512]

        logits = self.classifier(final_feat)
        return logits


# =====================================
# Train / Eval（改进版）
# =====================================
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in loader:
            logits = model(
                input_audio=batch["audio_values"].to(device),
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                use_text=batch["use_text"].to(device)
            )
            preds = logits.argmax(dim=1)
            labels = batch["labels"].to(device)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    return accuracy, all_preds, all_labels


def train_model(train_csv, val_csv, audio_root, batch_size=8, lr=2e-5, epochs=10, warmup_epochs=2):
    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

    train_ds = PodcastDataset(train_csv, audio_root, tokenizer, feature_extractor)
    val_ds = PodcastDataset(val_csv, audio_root, tokenizer, feature_extractor)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)

    num_classes = len(train_ds.label_map)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 使用改进的模型
    model = ImprovedAudioTextClassifier(num_classes=num_classes, freeze_pretrained=False).to(device)

    # 分组学习率
    pretrained_params = []
    new_params = []
    for name, param in model.named_parameters():
        if 'audio_encoder' in name or 'text_encoder' in name:
            pretrained_params.append(param)
        else:
            new_params.append(param)

    optimizer = AdamW([
        {'params': pretrained_params, 'lr': lr * 0.1},  # 预训练层用更小学习率
        {'params': new_params, 'lr': lr}
    ])

    # 添加学习率调度器
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)

    criterion = nn.CrossEntropyLoss()
    best_acc = 0
    patience = 5
    no_improve = 0

    print(f"\n🚀 Starting training...")
    print(f"Device: {device}")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}\n")

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in pbar:
            optimizer.zero_grad()

            logits = model(
                input_audio=batch["audio_values"].to(device),
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                use_text=batch["use_text"].to(device)
            )
            loss = criterion(logits, batch["labels"].to(device))

            if torch.isnan(loss) or torch.isinf(loss):
                print(f"\n⚠️ Warning: Invalid loss detected, skipping batch")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / len(train_loader)
        val_acc, _, _ = evaluate(model, val_loader, device)

        print(f">>> Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({
                'model_state_dict': model.state_dict(),
                'epoch': epoch,
                'val_acc': val_acc,
                'label_map': train_ds.label_map
            }, "best_multimodal_model.pt")
            print(f"✔ Saved best model (Acc: {val_acc:.4f})")
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"\n⚠️ Early stopping after {epoch+1} epochs (no improvement for {patience} epochs)")
                break

        scheduler.step()

    print(f"\n🎉 Training complete. Best Val Acc = {best_acc:.4f}")
    return model, train_ds.label_map


# =====================================
# Test函数
# =====================================
def test_model(model_path, test_csv, audio_root, label_map, batch_size=8):
    """在测试集上评估模型性能"""
    from sklearn.metrics import classification_report, confusion_matrix
    import numpy as np

    print("\n" + "="*50)
    print("📊 Testing on Test Set")
    print("="*50 + "\n")

    # 加载模型
    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

    test_ds = PodcastDataset(test_csv, audio_root, tokenizer, feature_extractor)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)

    num_classes = len(label_map)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ImprovedAudioTextClassifier(num_classes=num_classes, freeze_pretrained=False).to(device)

    # 加载最佳模型权重
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✔ Loaded best model from epoch {checkpoint['epoch']+1} (Val Acc: {checkpoint['val_acc']:.4f})\n")

    # 评估
    test_acc, all_preds, all_labels = evaluate(model, test_loader, device)

    # 反转 label_map 以获取类别名称
    idx_to_label = {v: k for k, v in label_map.items()}
    class_names = [idx_to_label[i] for i in range(num_classes)]

    # 打印总体结果
    print(f"{'='*50}")
    print(f"🎯 Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"{'='*50}\n")

    # 详细分类报告
    print("📋 Classification Report:")
    print("-" * 50)
    print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))

    # 混淆矩阵
    print("\n📊 Confusion Matrix:")
    print("-" * 50)
    cm = confusion_matrix(all_labels, all_preds)

    # 打印格式化的混淆矩阵
    header = "True\\Pred  " + "  ".join([f"{name[:8]:>8}" for name in class_names])
    print(header)
    print("-" * len(header))
    for i, row in enumerate(cm):
        row_str = f"{class_names[i][:10]:<10} " + "  ".join([f"{val:>8}" for val in row])
        print(row_str)

    # 每个类别的准确率
    print("\n📈 Per-Class Accuracy:")
    print("-" * 50)
    class_correct = cm.diagonal()
    class_total = cm.sum(axis=1)
    for i, name in enumerate(class_names):
        acc = class_correct[i] / class_total[i] if class_total[i] > 0 else 0
        print(f"{name:20s}: {acc:.4f} ({class_correct[i]:3d}/{class_total[i]:3d})")

    # 最容易混淆的类别对
    print("\n🔀 Most Confused Class Pairs:")
    print("-" * 50)
    confusion_pairs = []
    for i in range(num_classes):
        for j in range(num_classes):
            if i != j and cm[i, j] > 0:
                confusion_pairs.append((cm[i, j], class_names[i], class_names[j]))

    confusion_pairs.sort(reverse=True)
    for count, true_class, pred_class in confusion_pairs[:10]:
        print(f"{true_class:15s} → {pred_class:15s}: {count:3d} times")

    print("\n" + "="*50)
    print("✅ Testing Complete!")
    print("="*50)

    return test_acc, all_preds, all_labels


# =====================================
# Run training and testing
# =====================================
if __name__ == "__main__":
    datadir = "/content/drive/My Drive/CS441/FP2/data/"

    # 训练模型
    model, label_map = train_model(
        train_csv=os.path.join(datadir, "train_transcripts.csv"),
        val_csv=os.path.join(datadir, "val_transcripts.csv"),
        audio_root=datadir,
        batch_size=8,
        lr=2e-5,
        epochs=20,
        warmup_epochs=2
    )

    # 在测试集上评估
    test_acc, test_preds, test_labels = test_model(
        model_path="best_multimodal_model.pt",
        test_csv=os.path.join(datadir, "test_transcripts.csv"),
        audio_root=datadir,
        label_map=label_map,
        batch_size=8
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📊 Loaded 1080 samples, 6 classes: ['business', 'comedy', 'education', 'news', 'religion', 'sports']
📊 Loaded 240 samples, 6 classes: ['business', 'comedy', 'education', 'news', 'religion', 'sports']


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🚀 Starting training...
Device: cuda
Total parameters: 162,446,472
Trainable parameters: 119,919,240



Epoch 1/20: 100%|██████████| 135/135 [00:34<00:00,  3.96it/s, loss=1.7530]


...

>>> Epoch 14 | Train Loss: 0.8933 | Val Acc: 0.5917 | LR: 5.46e-07


Epoch 15/20: 100%|██████████| 135/135 [00:32<00:00,  4.18it/s, loss=1.1337]


>>> Epoch 15 | Train Loss: 0.9118 | Val Acc: 0.6000 | LR: 4.12e-07

⚠️ Early stopping after 15 epochs (no improvement for 5 epochs)

🎉 Training complete. Best Val Acc = 0.6000

📊 Testing on Test Set

📊 Loaded 240 samples, 6 classes: ['business', 'comedy', 'education', 'news', 'religion', 'sports']


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✔ Loaded best model from epoch 10 (Val Acc: 0.6000)

🎯 Test Accuracy: 0.7875 (78.75%)

📋 Classification Report:
--------------------------------------------------
              precision    recall  f1-score   support

    business     0.6981    0.9250    0.7957        40
      comedy     0.7105    0.6750    0.6923        40
   education     0.8000    0.7000    0.7467        40
        news     0.7097    0.5500    0.6197        40
    religion     0.9524    1.0000    0.9756        40
      sports     0.8537    0.8750    0.8642        40

    accuracy                         0.7875       240
   macro avg     0.7874    0.7875    0.7824       240
weighted avg     0.7874    0.7875    0.7824       240


📊 Confusion Matrix:
--------------------------------------------------
True\Pred  business    comedy  educatio      news  religion    sports
---------------------------------------------------------------------
business         37         0         2         1         0         0
comedy      